In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp

from PIL import Image
import os

In [2]:
#import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

Instructions for updating:
non-resource variables are not supported in the long term


Should take around 3 minutes to load

In [3]:
h_dim = 1000
v_dim = 750
print("Loading Data from Memory")

root = "Train"
label_flood_dir = os.path.join(root,'Labeled','Flooded','image')
label_nonflood_dir = os.path.join(root,'Labeled','Non-Flooded','image')
flooded_img = []
nonflooded_img = []

for file in os.listdir(label_flood_dir):
    image = Image.open(os.path.join(label_flood_dir, file))
    flooded_img.append(np.array(image.resize((h_dim,v_dim))))
    
for file in os.listdir(label_nonflood_dir):
    image = Image.open(os.path.join(label_nonflood_dir, file))
    nonflooded_img.append(np.array(image.resize((h_dim,v_dim))))
print("Loaded!")

Loading Data from Memory
Loaded!


In [4]:
print("Flooded Image Shape: {}".format(flooded_img[0].shape))
print("Non_Flooded Image Shape: {}".format(nonflooded_img[0].shape))

data_img = np.vstack((np.array(flooded_img), np.array(nonflooded_img))) / 255.
data_img.shape

Flooded Image Shape: (750, 1000, 3)
Non_Flooded Image Shape: (750, 1000, 3)


(398, 750, 1000, 3)

In [15]:
train_idx, test_idx, train_labels, test_labels = train_test_split(flooded_img, nonflooded_img)

Data len 51; n is 51 ;train size 40, test size 11
Data len 347; n is 347 ;train size 277, test size 70
Training Indices len 317
Testing Indices len 81


In [16]:
# convolutional layer
# x the input 
# shape is dimension of input
def conv_layer(x, shape):

    weights = tf.Variable(tf.truncated_normal(shape, stddev=0.05))
    bias = tf.Variable(tf.constant(0.05, shape=[shape[-1]]))

    out = tf.nn.conv2d(input=x, filters=weights, strides=[1,1,1,1], padding='SAME')
    out += bias
    return out

# pooling layer
def max_pool(x, k=2):

    out = tf.nn.max_pool(value=x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')
    return out

# fully connected layer
def fully_connected_layer(x, shape):

    weights = tf.Variable(tf.truncated_normal(shape, stddev=0.05))
    bias = tf.Variable(tf.constant(0.05, shape=[shape[1]]))

    out = tf.matmul(a=x, b=weights)
    out += bias
    return out

# flatten layer
def flatten_layer(x):
    
    size = x.get_shape()[1:4].num_elements()
    out = tf.reshape(x, [-1,size])
    return out, size

# relu
relu = lambda x: tf.nn.relu(features=x)

# softmax
softmax = lambda x: tf.nn.softmax(logits=x)

# sigmoid
sigmoid = lambda x: tf.nn.sigmoid(x)

# batch norm
batch_norm = lambda x: tf.layers.batch_normalization(x)

In [17]:
# define CNN
def toy_model(x):
    
    #shape = [filter_size, filter_size, num_input_channels, num_filters]
    def conv_block(x,in_channels,out_channels,kernel_h=5,kernel_w =5):
        shape = [kernel_h,kernel_w,in_channels,out_channels]
        x = conv_layer(x, shape)
        x = relu(x)
        x = batch_norm(x)
        x = max_pool(x, k=2)
        return x
    
    x = conv_block(x,3,16)
    x = conv_block(x,16,16)
    x = conv_block(x,16,32)
    x = conv_block(x,32,32)
    x = conv_block(x,32,64)
    x = conv_block(x,64,64)
    x = conv_block(x,64,1)

    # flatten output and put through a fully connected layer
    flat1, size1 = flatten_layer(x)
    fc1 = fully_connected_layer(flat1, [size1, 64])
    fc1 = relu(fc1)

    fc2 = fully_connected_layer(fc1, [64, 1])

    return fc2

In [18]:
# def sharpen(p):
#     T = 0.5
#     pred = p**(1./T)/(p**(1./T) + (1.-p)**(1./T))
#     return pred

In [19]:
# define inputs
x = tf.placeholder(tf.float32, [None, v_dim, h_dim, 3])
y = tf.placeholder(tf.float32, [None, 1])
y_train_true = np.array(train_labels).reshape(-1,1)
y_test_true = np.array(test_labels).reshape(-1,1)

# run model with placeholder tensors
pred = toy_model(x)

# sharpen
# pred = sharpen(pred)

# define loss
cross_entropy = tf.losses.sigmoid_cross_entropy(logits=pred, multi_class_labels=y)
cost = tf.reduce_mean(cross_entropy)

# define accuracy
pred_class = tf.round(sigmoid(pred))
pred_correct = tf.equal(pred_class, tf.cast(y, tf.float32))
accuracy = tf.reduce_mean(tf.cast(pred_correct, tf.float32))

# define optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=1e-2).minimize(cost)

# initialize variables
init = tf.global_variables_initializer()
training_iters = 500
batch_size = 4 #len(train_idx)

Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [20]:
# train model
with tf.Session(config=config) as sess:
    sess.run(init)
    train_loss = []
    test_loss = []
    train_accuracy = []
    test_accuracy = []
    best_acc = 0.
    summary_writer = tf.summary.FileWriter('./Output', sess.graph)
    num_batches = len(train_idx)//batch_size
    
    for i in range(training_iters):
        
        # Reset metrics
        loss_total = 0
        acc_total = 0
        train_results = []
   
        # Run optimization 
        # Calculate batch loss and accuracy
        for batch in range(num_batches):
            batch_x = data_img[train_idx,:,:,:][batch*batch_size:min((batch+1)*batch_size,len(train_idx))]
            batch_y = y_train_true[batch*batch_size:min((batch+1)*batch_size,len(y_train_true))]    

            feed_dict={x: batch_x, y: batch_y}
            opt = sess.run(optimizer, feed_dict=feed_dict)
            loss, acc, pred_labels = sess.run([cost, accuracy, pred_class], feed_dict=feed_dict)
            loss_total += loss
            acc_total += acc
            train_results.append(pred_labels)

        # Average metrics
        ave_loss = loss_total/num_batches
        ave_acc = acc_total/num_batches

        # Calculate accuracy for all test images
        valid_loss, test_acc, test_results = sess.run([cost, accuracy, pred_class],
                                feed_dict={x: data_img[test_idx,:,:,:], y : y_test_true})
        
        # Update metrics
        train_loss.append(ave_loss)
        test_loss.append(valid_loss)
        train_accuracy.append(ave_acc)
        test_accuracy.append(test_acc)
        if test_acc > best_acc:
            best_model_train_labels = tf.stack(tf.reshape(tf.stack(train_results),[-1,1])).eval()
            best_model_test_labels = test_results
            best_acc = test_acc
        
        # Print metrics
        print("Iter " + str(i) + ", Loss= " + \
                      "{:.6f}".format(ave_loss) + ", Training Accuracy= " + \
                      "{:.5f}".format(ave_acc)+ \
                      " ,Testing Accuracy:","{:.5f}".format(test_acc))
    summary_writer.close()

Iter 0, Loss= 0.673709, Training Accuracy= 0.70886 ,Testing Accuracy: 0.86420
Iter 1, Loss= 0.299032, Training Accuracy= 0.86076 ,Testing Accuracy: 0.86420
Iter 2, Loss= 0.513162, Training Accuracy= 0.87342 ,Testing Accuracy: 0.86420


KeyboardInterrupt: 

In [ ]:
best_model_train_labels

In [ ]:
best_model_test_labels

In [ ]:
for title, data in {"Loss":train_loss,"Train Accuracy": train_accuracy, "Test Accuracy": test_accuracy}.items():
    plt.plot(data)
    plt.title(title)
    plt.show()